HackerEarth Assessment EXL : Prateek Naharia


In [19]:
import pandas as pd
#/content/sample_submission.csv
#/content/test.csv
#/content/train.csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [7]:
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")
submission_df = pd.read_csv("/content/sample_submission.csv")
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Submission shape:", submission_df.shape)
print("train cols:", train_df.columns.tolist())
print("test cols:", test_df.columns.tolist())
print("submission cols:", submission_df.columns.tolist())

Train shape: (800, 6)
Test shape: (200, 5)
Submission shape: (200, 2)
train cols: ['text_id', 'text', 'category', 'word_count', 'readability_score', 'keywords']
test cols: ['text_id', 'text', 'word_count', 'readability_score', 'keywords']
submission cols: ['text_id', 'category']


In [15]:
train_df['full_text'] = train_df['text'] + ' ' + train_df['keywords']
test_df['full_text'] = test_df['text'] + ' ' + test_df['keywords']

In [16]:
train_df.head(1)

,text_id,text,category,word_count,readability_score,keywords,full_text
0,text_727,The patient's condition was characterized by m...,Musculoskeletal,114,63,"myalgia, fracture, tendon repair, tendon, spinal",The patient's condition was characterized by m...


In [41]:
test_df.head(5)

,text_id,text,word_count,readability_score,keywords,full_text
0,text_718,Follow-up evaluation for spinal showed persist...,114,62,"spinal, NSAIDs, back pain",Follow-up evaluation for spinal showed persist...
1,text_529,Medical history significant for glandular and ...,114,64,"glandular, levothyroxine, Cushing's syndrome, ...",Medical history significant for glandular and ...
2,text_35,Follow-up evaluation for atherosclerosis showe...,114,62,"cyanosis, atherosclerosis, bypass surgery",Follow-up evaluation for atherosclerosis showe...
3,text_549,A 74-year-old female presents with polydipsia ...,114,60,"exercise therapy, glandular, polydipsia, adren...",A 74-year-old female presents with polydipsia ...
4,text_676,A 88-year-old male presents with back pain and...,114,62,"swelling, back pain, osteoporosis, joint repla...",A 88-year-old male presents with back pain and...


In [17]:
pip install pandas scikit-learn

In [43]:
#encode lables
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['category'])
train_df.head(5)

,text_id,text,category,word_count,readability_score,keywords,full_text,label
0,text_727,The patient's condition was characterized by m...,Musculoskeletal,114,63,"myalgia, fracture, tendon repair, tendon, spinal",The patient's condition was characterized by m...,5
1,text_538,Follow-up evaluation for glandular showed pers...,Endocrine,114,63,"levothyroxine, glandular, fatigue",Follow-up evaluation for glandular showed pers...,2
2,text_902,Patient presents with myalgia and fatigue. Ini...,Infectious,114,59,"antifungals, myalgia, contagious, fatigue, fungal",Patient presents with myalgia and fatigue. Ini...,4
3,text_251,Follow-up evaluation for hepatic showed persis...,Gastrointestinal,114,62,"proton pump inhibitors, constipation, hepatic",Follow-up evaluation for hepatic showed persis...,3
4,text_919,A 77-year-old male with a history of malaria w...,Infectious,114,63,"antifungals, lymphadenopathy, sore throat, pro...",A 77-year-old male with a history of malaria w...,4


In [35]:
# 4. Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    train_df['full_text'],
    train_df['label'],
    test_size=0.2,
    stratify=train_df['label'],
    random_state=42
)

In [37]:
#TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000)

In [38]:
y_val_pred = clf.predict(X_val_vec)
f1 = f1_score(y_val, y_val_pred, average='weighted')
print(f"Validation Weighted F1 Score: {f1 * 100:.2f}")

Validation Weighted F1 Score: 88.81


In [39]:
X_test_vec = vectorizer.transform(test_df['full_text'])
test_preds = clf.predict(X_test_vec)
test_labels = le.inverse_transform(test_preds)
submission_df = pd.DataFrame({
    'text_id': test_df['text_id'],
    'category': test_labels
})
submission_df.to_csv("submission.csv", index=False)
print("submissioncreated")

submissioncreated
